Proyecto EXANI
El presente proyecto, presenta los resultados de un examen de ingreso a educación

Curso INTRODUCCIÓN A LA CIENCIA DE DATOS E INTELIGENCIA ARTIFICIAL

Instructor: Dr. Dan Santivañez

Organización: Haciendo Ciencia

Proyecto Final Elaborado por Sindya Yadira Castillo Ortiz

Introducción en el presente proyecto se analizan los datos inherentes a los resultados de la aplicación de un examen de ingreso a educación superior del Instituto Tecnológico de Apizaco del año 2017, como un punto de inicio del estudio.
El Estadio tiene por objetivo el analizar la posibilidad de predicción de resultado de desempeño, considerando algunas de las variables existentes. Teniendo como variable objetivo el Desemepeño alcanzado (class).
En el Dataframe se encontrarán variables relacionadas con el sexo, año de nacimiento, las habilidades de los estudiantes, el promedio de educación media superior, si contaron con beca, elementos del perfil socioeconómico, entre otros.

PASO 1: Cargar librerias

Entre las librerias más empleadas estan:
Pandas: Biblioteca esencial para manipulación de datos, usada para cargar, limpiar y transformar el dataset.

Matplotlib: Biblioteca fundamental de visualización en Python que permite crear gráficos personalizados como histogramas, gráficos de barras y diagramas de dispersión. Ofrece control detallado sobre aspectos como ejes, colores y etiquetas, siendo ideal para visualizaciones básicas y avanzadas.

Seaborn: Construida sobre Matplotlib, simplifica la creación de gráficos estadísticos con una sintaxis más intuitiva y estilos visuales atractivos por defecto. Es excelente para análisis univariados y bivariados (e.g., boxplots, heatmaps).

Ydata-Profiling: Herramienta de automatización que genera reportes detallados del dataset, útil para una exploración inicial rápida.


In [15]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE

# 1. Instalamos librerías necesarias para trabajar con Google Sheets
!pip install --upgrade gspread gspread-dataframe

# 2. Importamos librerías de Google
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from oauth2client.client import GoogleCredentials

PASO 2 INSTALAR COMPONENTES
Es necesario considerar que entornos de Colab es necesario instalar algunos componentes

In [16]:
!pip install openpyxl
!pip uninstall pandas_profiling -y
!pip install ydata-profiling

PASO 3 Cargar los datos

In [17]:
df = pd.read_csv('/content/EXANIVF.csv')
                  # engine= 'openpyxl') # Changed the engine to 'openpyxl'

PRIMER PROCESO EDA (Análisis Exploratorio de Datos) Para este proceso de pretende:

Es el proceso inicial de análisis de un conjunto de datos para:

Comprender su estructura

Detectar patrones

Identificar valores atípicos (outliers)

Revisar la calidad de los datos (datos faltantes, duplicados, errores)

Observar relaciones entre variables


In [18]:
#Mostrar Dataframe
df.head()

,ANO_NAC,SEXO,ENTI_NAC,LI_MAD,LI_PAD,EDO_PROC,PROM_BAC,BEC_NEC,EXA_EXT,DES_DESA,DES_ESME,DES_TERM,DES_DURO,ESCO_MAD,ESCO_PAD,CUAN_LIB,SER_LAV,SER_REF,SER_HOR,SER_INTE,SER_CABL,BIEN_PC,SER_TV,SER_AUTO,SER_BANO,VAC_RM,class
0,1996,1,21,2,2,29,7.0,1,1,1,2,3,3,6,6,5,2,1,2,2,2,1,2,2,2,1,NO SUFICIENTE
1,1996,2,29,2,2,29,9.3,1,1,5,5,5,5,2,2,2,2,1,2,2,2,1,2,1,2,1,NO SUFICIENTE
2,1996,1,29,2,2,29,8.6,2,1,1,3,4,4,4,2,3,1,1,1,1,1,3,4,2,2,3,NIVEL SATISFACTORIO
3,1995,1,29,2,2,29,9.8,2,1,1,5,5,5,6,5,6,1,1,2,2,2,2,3,1,1,2,NIVEL SOBRESALIENTE
4,1996,1,29,1,1,29,7.3,1,2,2,4,5,4,3,3,3,2,1,2,2,2,2,3,2,2,1,NIVEL SATISFACTORIO


PASO 1 EDA Identificar la estructura de los datos del Dataframe, para poder identificar los tipos de datos, la existencia de nulos y si los valores del TARGET estan balanceados.
***Es conocido como TARGET la variable objetivo de nuestro estudio

In [19]:
print(df.shape)
df.head()
df.info()     # tipos y nulos
df['class'].value_counts()  # ¿hay desbalance?


(813, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ANO_NAC   813 non-null    int64  
 1   SEXO      813 non-null    int64  
 2   ENTI_NAC  813 non-null    int64  
 3   LI_MAD    813 non-null    int64  
 4   LI_PAD    813 non-null    int64  
 5   EDO_PROC  813 non-null    int64  
 6   PROM_BAC  813 non-null    float64
 7   BEC_NEC   813 non-null    int64  
 8   EXA_EXT   813 non-null    int64  
 9   DES_DESA  813 non-null    int64  
 10  DES_ESME  813 non-null    int64  
 11  DES_TERM  813 non-null    int64  
 12  DES_DURO  813 non-null    int64  
 13  ESCO_MAD  813 non-null    int64  
 14  ESCO_PAD  813 non-null    int64  
 15  CUAN_LIB  813 non-null    int64  
 16  SER_LAV   813 non-null    int64  
 17  SER_REF   813 non-null    int64  
 18  SER_HOR   813 non-null    int64  
 19  SER_INTE  813 non-null    int64  
 20  SER_CABL  813 non-null

,count
class,
NO SUFICIENTE,390
NIVEL SATISFACTORIO,372
NIVEL SOBRESALIENTE,51


Nota: Derivado del analisis de balance, podemos definir que es necesario aplicar un ajuste, creando nuevos casos sinteticos, para balancear el proceso y evitar sobreflujo o una tendencia creada.

PASO 2 EDA: Es neecsario transformar los datos en categorias para poder aplicar los modelos de predicción. Dado que los datos que tenemos son numéricos

In [20]:
# Conversión de tipos de datos

# Lista de columnas a convertir
cols_to_convert = ['SEXO', 'EDO_PROC', 'DES_DURO', 'ESCO_PAD', 'SER_INTE', 'BIEN_PC', 'SER_AUTO']

# Asegurar que no hay valores nulos
assert df[cols_to_convert].isna().sum().sum() == 0, "Hay valores nulos en las columnas"

# Convertir a enteros (asumiendo que las columnas tienen valores mapeables a enteros)
# Si las columnas tienen strings, usar un mapeo o codificación previa
for col in cols_to_convert:
    # Ejemplo: si son strings, codificar a enteros (ajustar según los datos reales)
    if df[col].dtype == 'object':
        unique_values = df[col].unique()
        mapping = {val: idx for idx, val in enumerate(unique_values)}
        df[col] = df[col].map(mapping).astype('int32')
    else:
        df[col] = df[col].astype('int32')

# Convertir todas las columnas a category de una vez
df[cols_to_convert] = df[cols_to_convert].astype('category')

# Verificar los tipos de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ANO_NAC   813 non-null    int64   
 1   SEXO      813 non-null    category
 2   ENTI_NAC  813 non-null    int64   
 3   LI_MAD    813 non-null    int64   
 4   LI_PAD    813 non-null    int64   
 5   EDO_PROC  813 non-null    category
 6   PROM_BAC  813 non-null    float64 
 7   BEC_NEC   813 non-null    int64   
 8   EXA_EXT   813 non-null    int64   
 9   DES_DESA  813 non-null    int64   
 10  DES_ESME  813 non-null    int64   
 11  DES_TERM  813 non-null    int64   
 12  DES_DURO  813 non-null    category
 13  ESCO_MAD  813 non-null    int64   
 14  ESCO_PAD  813 non-null    category
 15  CUAN_LIB  813 non-null    int64   
 16  SER_LAV   813 non-null    int64   
 17  SER_REF   813 non-null    int64   
 18  SER_HOR   813 non-null    int64   
 19  SER_INTE  813 non-null    category
 20  SER_CABL  

In [21]:
df.head()

,ANO_NAC,SEXO,ENTI_NAC,LI_MAD,LI_PAD,EDO_PROC,PROM_BAC,BEC_NEC,EXA_EXT,DES_DESA,DES_ESME,DES_TERM,DES_DURO,ESCO_MAD,ESCO_PAD,CUAN_LIB,SER_LAV,SER_REF,SER_HOR,SER_INTE,SER_CABL,BIEN_PC,SER_TV,SER_AUTO,SER_BANO,VAC_RM,class
0,1996,1,21,2,2,29,7.0,1,1,1,2,3,3,6,6,5,2,1,2,2,2,1,2,2,2,1,NO SUFICIENTE
1,1996,2,29,2,2,29,9.3,1,1,5,5,5,5,2,2,2,2,1,2,2,2,1,2,1,2,1,NO SUFICIENTE
2,1996,1,29,2,2,29,8.6,2,1,1,3,4,4,4,2,3,1,1,1,1,1,3,4,2,2,3,NIVEL SATISFACTORIO
3,1995,1,29,2,2,29,9.8,2,1,1,5,5,5,6,5,6,1,1,2,2,2,2,3,1,1,2,NIVEL SOBRESALIENTE
4,1996,1,29,1,1,29,7.3,1,2,2,4,5,4,3,3,3,2,1,2,2,2,2,3,2,2,1,NIVEL SATISFACTORIO


In [22]:
df['ANO_NAC'].value_counts()

,count
ANO_NAC,
1996,413
1995,262
1994,74
1993,24
1992,10
1991,9
1990,6
1985,3
1987,3


Nota: Dada la gran variabilidad de datos de año de nacimiento, es recomendable estratificarlo en dos grandes grupos: Menores de 29 y mayores de 29.

In [23]:
import time
# Convertir la estructura de dato de ANO_NAC a una categoría
ano_actual = time.localtime().tm_year  # Se extrae el año actual

# Calcular la edad restando el año de nacimiento al año actual
df['EDAD'] = ano_actual - df['ANO_NAC']

# Crear categorías basadas en la edad: =<29 y >29
df['CATEGORIA_EDAD'] = pd.cut(df['EDAD'],
                              bins=[-float('inf'), 29, float('inf')],
                              labels=['1', '2'], # 1:=<29, >29
                              include_lowest=True)

In [24]:
df['CATEGORIA_EDAD'].value_counts()

,count
CATEGORIA_EDAD,
1,413
2,400


In [25]:
cols = ['CATEGORIA_EDAD'] + [col for col in df.columns if col != 'CATEGORIA_EDAD']
df = df[cols]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 813 entries, 0 to 812
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   CATEGORIA_EDAD  813 non-null    category
 1   ANO_NAC         813 non-null    int64   
 2   SEXO            813 non-null    category
 3   ENTI_NAC        813 non-null    int64   
 4   LI_MAD          813 non-null    int64   
 5   LI_PAD          813 non-null    int64   
 6   EDO_PROC        813 non-null    category
 7   PROM_BAC        813 non-null    float64 
 8   BEC_NEC         813 non-null    int64   
 9   EXA_EXT         813 non-null    int64   
 10  DES_DESA        813 non-null    int64   
 11  DES_ESME        813 non-null    int64   
 12  DES_TERM        813 non-null    int64   
 13  DES_DURO        813 non-null    category
 14  ESCO_MAD        813 non-null    int64   
 15  ESCO_PAD        813 non-null    category
 16  CUAN_LIB        813 non-null    int64   
 17  SER_LAV         

In [28]:
# Eliminamos las columnas modificadas
df.drop(columns=['ANO_NAC', 'EDAD'], inplace=True)

In [29]:
df.head()

,CATEGORIA_EDAD,SEXO,ENTI_NAC,LI_MAD,LI_PAD,EDO_PROC,PROM_BAC,BEC_NEC,EXA_EXT,DES_DESA,DES_ESME,DES_TERM,DES_DURO,ESCO_MAD,ESCO_PAD,CUAN_LIB,SER_LAV,SER_REF,SER_HOR,SER_INTE,SER_CABL,BIEN_PC,SER_TV,SER_AUTO,SER_BANO,VAC_RM,class
0,1,1,21,2,2,29,7.0,1,1,1,2,3,3,6,6,5,2,1,2,2,2,1,2,2,2,1,NO SUFICIENTE
1,1,2,29,2,2,29,9.3,1,1,5,5,5,5,2,2,2,2,1,2,2,2,1,2,1,2,1,NO SUFICIENTE
2,1,1,29,2,2,29,8.6,2,1,1,3,4,4,4,2,3,1,1,1,1,1,3,4,2,2,3,NIVEL SATISFACTORIO
3,2,1,29,2,2,29,9.8,2,1,1,5,5,5,6,5,6,1,1,2,2,2,2,3,1,1,2,NIVEL SOBRESALIENTE
4,1,1,29,1,1,29,7.3,1,2,2,4,5,4,3,3,3,2,1,2,2,2,2,3,2,2,1,NIVEL SATISFACTORIO


### Transformación de datos

Categóricos -> Dummies

Numéricos -> Estandarizamos con Z-score

In [30]:
# Codificar la variable target
le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])

In [31]:
df['class'].value_counts()

,count
class,
2,390
0,372
1,51


In [42]:
# Definir variables categóricas vs. numéricas
categorical_cols = ['CATEGORIA_EDAD', 'SEXO', 'EDO_PROC', 'DES_DURO', 'ESCO_PAD', 'SER_INTE', 'BIEN_PC', 'SER_AUTO']
numerical_cols = ['PROM_BAC']

In [43]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Preprocesamiento con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

In [44]:
# Dividir los datos
# Dividir datos
X = df.drop('class', axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Aplicar transformaciones
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [45]:
y_train.value_counts()

,count
class,
2,312
0,297
1,41


### Aplicar una técnica cuando hay desbalance

SMOTE (Synthetic Minority Oversampling Technique) es una técnica para balancear conjuntos de datos desbalanceados. Genera muestras sintéticas de la clase minoritaria interpolando entre ejemplos existentes, en lugar de replicarlos. Sirve para mejorar el rendimiento de modelos de machine learning al evitar el sesgo hacia la clase mayoritaria, aumentando la representación de la clase minoritaria sin sobreajuste.

In [46]:
from imblearn.over_sampling import SMOTE
# Verificar balance
print(y_train.value_counts())

# Aplicar SMOTE si hay desbalance
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_preprocessed, y_train)

class
2    312
0    297
1     41
Name: count, dtype: int64


In [47]:
y_train_balanced.value_counts()

,count
class,
0,312
2,312
1,312


### Modelos

In [48]:
rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train_balanced, y_train_balanced)
best_rf = grid_search.best_estimator_

In [49]:
svm = SVC(random_state=42)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

grid_search_svm = GridSearchCV(svm, param_grid, cv=3, scoring='f1_weighted')
grid_search_svm.fit(X_train_balanced, y_train_balanced)
best_svm = grid_search_svm.best_estimator_

In [50]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    X_train_preprocessed, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.4449 - loss: 0.9532 - val_accuracy: 0.5154 - val_loss: 0.8601
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5490 - loss: 0.8446 - val_accuracy: 0.5154 - val_loss: 0.8516
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.5543 - loss: 0.8341 - val_accuracy: 0.5385 - val_loss: 0.8529
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6231 - loss: 0.8079 - val_accuracy: 0.5462 - val_loss: 0.8431
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.5915 - loss: 0.8108 - val_accuracy: 0.5231 - val_loss: 0.8459
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5820 - loss: 0.8159 - val_accuracy: 0.5308 - val_loss: 0.8436
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6263 - loss: 0.7623 - val_accuracy: 0.5308 - val_loss: 0.8484
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5708 - loss: 0.7967 - val_accuracy: 0.5385 - 

### Evaluar

In [51]:
# Función para evaluar
def evaluate_model(model, X_test, y_test, is_ann=False):
    if is_ann:
        y_pred = np.argmax(model.predict(X_test), axis=1)
    else:
        y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))

# Evaluar Random Forest
evaluate_model(best_rf, X_test_preprocessed, y_test)

# Evaluar SVM
evaluate_model(best_svm, X_test_preprocessed, y_test)

# Evaluar ANN
evaluate_model(model, X_test_preprocessed, y_test, is_ann=True)

              precision    recall  f1-score   support

           0       0.47      0.49      0.48        75
           1       0.00      0.00      0.00        10
           2       0.51      0.54      0.52        78

    accuracy                           0.48       163
   macro avg       0.33      0.34      0.34       163
weighted avg       0.46      0.48      0.47       163

Accuracy: 0.48466257668711654
              precision    recall  f1-score   support

           0       0.51      0.47      0.49        75
           1       0.23      0.30      0.26        10
           2       0.51      0.53      0.52        78

    accuracy                           0.48       163
   macro avg       0.41      0.43      0.42       163
weighted avg       0.49      0.48      0.49       163

Accuracy: 0.48466257668711654
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
              precision    recall  f1-score   support

           0       0.55      0.47      0.50        75
           1       0.00      0.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Conclusiones:
Derivado de la obtención de un Accuracy de 0.558282208588957, se puede concluir que el modelo, si bien puede ser aceptable, dado que es mayor de .5, este puede ser no muy confiable. Por lo cual es necesario considerar, realizar ajustes en el proceso, probando diferentes acciones como:

Revisar balance de clases.

Aplicar otras métricas (confusion matrix, F1-score, etc.).

Ajustar el modelo: parámetros, features, algoritmos.

Aplicar técnicas de preprocesamiento (normalización, reducción de ruido).

Probar otros modelos o hacer ensambles.

